In [ ]:
CITY_GRAPH_FILES = {
    "Boston": [
        "Boston_Massachusetts_USA.graphml",
    ],
    "Chicago": [
        "Chicago.graphml",
        "Chicago_Illinois.graphml",
        "Chicago_Illinois_USA.graphml",
    ],
    "Dallas": [
        "Dallas_Texas_USA.graphml",
    ],
    "Phoenix": [
        "Phoenix_Arizona_USA.graphml",
    ],
    "Pittsburgh": [
        "Pittsburgh.graphml",
        "Pittsburgh_Pennsylvania.graphml",
        "Pittsburgh_Pennsylvania_USA.graphml",
        "Pittsburgh_Allegheny_County_Pennsylvania_United_States.graphml",
    ],
    "San Francisco": [
        "San_Francisco_California_USA.graphml",
    ],
}


In [ ]:
import os
import random
import numpy as np
import pandas as pd
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt

# -----------------------------
# GLOBAL SETTINGS
# -----------------------------
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

ox.settings.log_console = False
ox.settings.use_cache = False   # we are NOT using Overpass at all

GRAPH_DIR = "/Users/surabhiraghavan/Documents/Sem 3/Network Science and Analysis/urban-mobility/backend/graphs"

# -----------------------------
# GRAPH FILE MAPPING
# -----------------------------
CITY_GRAPH_FILES = {
    "Boston": [
        "Boston.graphml",
        "Boston_Massachusetts_USA.graphml",
        "Boston_Suffolk_County_Massachusetts_United_States.graphml",
    ],
    "Chicago": [
        "Chicago.graphml",
        "Chicago_Illinois.graphml",
        "Chicago_Illinois_USA.graphml",
    ],
    "Dallas": [
        "Dallas_Texas_USA.graphml",
    ],
    "Phoenix": [
        "Phoenix_Arizona_USA.graphml",
    ],
    "Pittsburgh": [
        "Pittsburgh.graphml",
        "Pittsburgh_Pennsylvania.graphml",
        "Pittsburgh_Pennsylvania_USA.graphml",
        "Pittsburgh_Allegheny_County_Pennsylvania_United_States.graphml",
    ],
    "San Francisco": [
        "San_Francisco_California_USA.graphml",
    ],
}

# -----------------------------
# GRAPH LOADER (GRAPHML ONLY)
# -----------------------------
def load_city_graph(city):
    city_key = city.lower().replace(" ", "_")

    candidates = []
    for fname in os.listdir(GRAPH_DIR):
        if not fname.endswith(".graphml"):
            continue

        f_lower = fname.lower()

        # must contain city name
        if city_key not in f_lower:
            continue

        # exclude known false positives
        if "county" in f_lower and city_key not in f_lower.split("_county")[0]:
            continue

        candidates.append(fname)

    if not candidates:
        raise FileNotFoundError(f"No GraphML file found for {city}")

    # Prefer shorter, cleaner filenames
    candidates.sort(key=len)
    chosen = candidates[0]
    path = os.path.join(GRAPH_DIR, chosen)

    print(f"Loading graph for {city}: {chosen}")
    G = ox.load_graphml(path)

    # ensure projected
    if "crs" not in G.graph or G.graph["crs"] is None:
        G = ox.project_graph(G)

    # ensure travel time
    u, v, k, data = list(G.edges(keys=True, data=True))[0]
    if "travel_time" not in data:
        G = ox.add_edge_speeds(G)
        G = ox.add_edge_travel_times(G)
        ox.save_graphml(G, path)

    return G


# -----------------------------
# BASELINE NETWORK STATISTICS
# -----------------------------
def compute_baseline_stats(G):
    Gu = nx.Graph(G)

    stats = {}
    stats["nodes"] = Gu.number_of_nodes()
    stats["edges"] = Gu.number_of_edges()
    stats["density"] = nx.density(Gu)

    degrees = np.array([d for _, d in Gu.degree()])
    stats["avg_degree"] = degrees.mean()

    # approximate ASPL (sampled)
    sample = random.sample(list(Gu.nodes()), min(500, Gu.number_of_nodes()))
    subG = Gu.subgraph(sample)

    # take largest connected component
    if not nx.is_connected(subG):
        largest_cc = max(nx.connected_components(subG), key=len)
        subG = subG.subgraph(largest_cc)

    stats["aspl"] = nx.average_shortest_path_length(subG)


    stats["clustering"] = nx.average_clustering(Gu)
    stats["assortativity"] = nx.degree_assortativity_coefficient(Gu)

    bridges = list(nx.bridges(Gu))
    stats["bridge_fraction"] = len(bridges) / Gu.number_of_edges()

    return stats

# -----------------------------
# FAILURE MODELS
# -----------------------------
def select_edges(G, strategy, severity):
    edges = list(G.edges(keys=True))
    k = int(len(edges) * severity)

    if strategy == "random":
        return random.sample(edges, k)

    if strategy == "targeted":
        Gu = nx.Graph(G)
        bc = nx.edge_betweenness_centrality(Gu, k=200, seed=RANDOM_SEED)
        ranked = sorted(bc.items(), key=lambda x: x[1], reverse=True)
        return [(u, v, 0) for (u, v), _ in ranked[:k]]

    raise ValueError("Unknown strategy")

# -----------------------------
# OD PAIRS
# -----------------------------
def sample_od_pairs(G, n_pairs=200):
    nodes = list(G.nodes())
    return [tuple(random.sample(nodes, 2)) for _ in range(n_pairs)]

# -----------------------------
# SIMULATION ENGINE
# -----------------------------
def simulate_failure(G, strategy, severity, n_pairs=200):
    G0 = G.copy()
    Gf = G.copy()

    Gf.remove_edges_from(select_edges(G, strategy, severity))

    od_pairs = sample_od_pairs(G0, n_pairs)

    ratios = []
    disconnected = 0

    for o, d in od_pairs:
        try:
            t0 = nx.shortest_path_length(G0, o, d, weight="travel_time")
            tf = nx.shortest_path_length(Gf, o, d, weight="travel_time")
            ratios.append(tf / t0)
        except nx.NetworkXNoPath:
            disconnected += 1

    avg_ratio = np.mean(ratios) if ratios else np.inf
    pct_disconnected = disconnected / n_pairs

    resilience = np.exp(-avg_ratio) * (1 - pct_disconnected)

    return avg_ratio, pct_disconnected, resilience

# -----------------------------
# LOAD ALL GRAPHS
# -----------------------------
cities = list(CITY_GRAPH_FILES.keys())
graphs = {}
baseline_rows = []

for city in cities:
    G = load_city_graph(city)
    graphs[city] = G

    row = compute_baseline_stats(G)
    row["city"] = city
    baseline_rows.append(row)

baseline_df = pd.DataFrame(baseline_rows)
baseline_df.to_csv("baseline_network_stats.csv", index=False)

print("\nBaseline Network Statistics")
print(baseline_df)

# -----------------------------
# RUN SIMULATIONS
# -----------------------------
severities = [0.05, 0.1, 0.2, 0.3]
scenarios = ["random", "targeted"]

results = []

for city, G in graphs.items():
    for scenario in scenarios:
        for s in severities:
            avg_ratio, pct_disc, res = simulate_failure(G, scenario, s)
            results.append({
                "city": city,
                "scenario": scenario,
                "severity": s,
                "avg_ratio": avg_ratio,
                "pct_disconnected": pct_disc,
                "resilience": res
            })

results_df = pd.DataFrame(results)
results_df.to_csv("simulation_results.csv", index=False)

# -----------------------------
# TABLE: SCENARIO SUMMARY
# -----------------------------
scenario_summary = (
    results_df
    .groupby("scenario")["resilience"]
    .agg(["mean", "std"])
)

print("\nScenario-wise Resilience")
print(scenario_summary)

# -----------------------------
# PLOT: ROBUSTNESS CURVES
# -----------------------------
plt.figure(figsize=(8, 5))
for city in cities:
    vals = results_df[
        (results_df.city == city) &
        (results_df.scenario == "random")
    ].sort_values("severity")

    plt.plot(vals.severity, vals.resilience, marker="o", label=city)

plt.xlabel("Severity (fraction removed)")
plt.ylabel("Resilience")
plt.title("Resilience vs Severity (Random Failure)")
plt.legend()
plt.tight_layout()
plt.show()

# -----------------------------
# PLOT: DISCONNECTED OD PAIRS
# -----------------------------
plt.figure(figsize=(8, 5))
for city in cities:
    vals = results_df[
        (results_df.city == city) &
        (results_df.scenario == "random")
    ].sort_values("severity")

    plt.plot(vals.severity, vals.pct_disconnected, marker="o", label=city)

plt.xlabel("Severity")
plt.ylabel("Fraction Disconnected OD Pairs")
plt.title("Connectivity Collapse vs Severity")
plt.legend()
plt.tight_layout()
plt.show()

# -----------------------------
# RANDOM VS TARGETED (CASE STUDY)
# -----------------------------
city = "Pittsburgh"
plt.figure(figsize=(7, 5))

for scenario in scenarios:
    vals = results_df[
        (results_df.city == city) &
        (results_df.scenario == scenario)
    ].sort_values("severity")

    plt.plot(vals.severity, vals.resilience, marker="o", label=scenario)

plt.xlabel("Severity")
plt.ylabel("Resilience")
plt.title(f"{city}: Random vs Targeted Failure")
plt.legend()
plt.tight_layout()
plt.show()

# -----------------------------
# FEATURE–RESILIENCE CORRELATION
# -----------------------------
merged = results_df.merge(baseline_df, on="city")

features = [
    "density",
    "avg_degree",
    "aspl",
    "clustering",
    "assortativity",
    "bridge_fraction"
]

corrs = {
    f: merged[f].corr(merged["resilience"])
    for f in features
}

corr_df = pd.DataFrame.from_dict(corrs, orient="index", columns=["correlation"])
corr_df.to_csv("feature_resilience_correlations.csv")

print("\nFeature–Resilience Correlations")
print(corr_df.sort_values("correlation"))

# -----------------------------
# ASSORTATIVITY VS RESILIENCE
# -----------------------------
plt.figure(figsize=(7, 5))
for city in cities:
    x = baseline_df.loc[baseline_df.city == city, "assortativity"].values[0]
    y = results_df.loc[results_df.city == city, "resilience"].mean()
    plt.scatter(x, y, s=80)
    plt.text(x, y, city, fontsize=9)

plt.xlabel("Assortativity")
plt.ylabel("Mean Resilience")
plt.title("Assortativity vs Urban Resilience")
plt.tight_layout()
plt.show()


Loading graph for Boston: Boston.graphml
Loading graph for Chicago: Chicago_Illinois.graphml
Loading graph for Dallas: Dallas_Texas_USA.graphml
Loading graph for Phoenix: Phoenix_Arizona_USA.graphml
Loading graph for Pittsburgh: Pittsburgh.graphml
Loading graph for San Francisco: San_Francisco_California_USA.graphml

Baseline Network Statistics
   nodes  edges   density  avg_degree      aspl  clustering  assortativity  \
0  11408  16618  0.000255    2.913394  1.333333    0.048467       0.127018   
1  29362  48731  0.000113    3.319324  1.000000    0.026053       0.197730   
2  36544  56723  0.000085    3.104367  1.333333    0.048720       0.176540   
3  48456  67262  0.000057    2.776209  1.000000    0.050153       0.093789   
4   9122  12988  0.000312    2.847621  1.333333    0.040665       0.101622   
5  10012  16393  0.000327    3.274670  1.500000    0.054249       0.216187   

   bridge_fraction           city  
0         0.103803         Boston  
1         0.047731        Chicago 